# Equation of state (EOS) WorkGraph

To run this tutorial, you need to install aiida-workgraph and restart the daemon. Open a terminal and run:
```console
pip install aiida-workgraph
verdi daemon restart
```

## Create the calcfunction node


### Visualize the workgraph

In [2]:
from workgraph_collections.cp2k.eos import eos_workgraph
from aiida import load_profile

load_profile()

wg = eos_workgraph("eos")
wg.to_html()

## Prepare inputs and run


In [3]:
from aiida import load_profile
from aiida.orm import Dict, KpointsData, StructureData, load_code, load_group
from ase.build import bulk

load_profile()

si = orm.StructureData(ase=bulk("Si"))
code = load_code("qe-7.2-pw@localhost")
pw_paras = Dict({
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 30,
            "ecutrho": 240,
            "occupations": "smearing",
            "smearing": "gaussian",
            "degauss": 0.1,
        },
    })
# Load the pseudopotential family.
pseudo_family = load_group('SSSP/1.3/PBEsol/efficiency')
pseudos = pseudo_family.get_pseudos(structure=si)
#
metadata = {'options': {
                'resources': {
                'num_machines': 1,
                'num_mpiprocs_per_machine': 1,
            },
        }
    }

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
pseudos = pseudo_family.get_pseudos(structure=si)
scf_inputs = {"code": code,
        "parameters": pw_paras,
        "kpoints": kpoints,
        "pseudos": pseudos,
        "metadata": metadata
        }
#-------------------------------------------------------
# set the input parameters for each node
wg.nodes["scale_structure1"].set({"structure": si, "scales": [0.95, 1.0, 1.05]})
wg.nodes["all_scf1"].set({"scf_inputs": scf_inputs})
print("Waiting for the workgraph to finish...")
wg.submit(wait=True, timeout=300)
# one can also run the workgraph directly
# wg.run()



Waiting for the workgraph to finish...
WorkGraph node created, PK: 14034


Print out the results:

In [4]:
data = wg.nodes["eos1"].outputs["result"].value.get_dict()
print('B: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**data))

B: 0.53596259211164
v0: 41.134100879971
e0: -308.19240692174
v0: 41.134100879971
